In [1]:
import nltk

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv("Train.csv")

In [4]:
train.head()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,not happy
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,not happy
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,not happy
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,happy
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,not happy


In [142]:
test = pd.read_csv("Test.csv")

In [6]:
test.head()

,User_ID,Description,Browser_Used,Device_Used
0,id80132,Looking for a motel in close proximity to TV t...,Firefox,Mobile
1,id80133,Walking distance to Madison Square Garden and ...,InternetExplorer,Desktop
2,id80134,Visited Seattle on business. Spent - nights in...,IE,Tablet
3,id80135,This hotel location is excellent and the rooms...,Edge,Mobile
4,id80136,This hotel is awesome I love the service Antho...,Mozilla,Mobile


In [7]:
df =train

In [8]:
df["Is_Response"].value_counts()

happy        26521
not happy    12411
Name: Is_Response, dtype: int64

In [9]:
df.drop(columns = ['User_ID', 'Browser_Used', 'Device_Used'],inplace = True)
df.head()

,Description,Is_Response
0,The room was kind of clean but had a VERY stro...,not happy
1,I stayed at the Crown Plaza April -- - April -...,not happy
2,I booked this hotel through Hotwire at the low...,not happy
3,Stayed here with husband and sons on the way t...,happy
4,My girlfriends and I stayed here to celebrate ...,not happy


In [12]:
stop_words = set(nltk.corpus.stopwords.words('english'))

In [40]:
example_text = "Paragraph Writing: Take a wild guess as to what will you say to describe a paragraph? From whatever you may know so far, many students describe paragraphs as what consists the story, or a set of sentences that are grouped together to form a paragraph or a set of sentences that cover half a page of your story, and so on. https://www.twitter.com"

process_text(example_text)

<function str.strip(chars=None, /)>

In [46]:
data.head()

,Description,Is_Response,clean_text
0,The room was kind of clean but had a VERY stro...,not happy,<built-in method strip of str object at 0x0000...
1,I stayed at the Crown Plaza April -- - April -...,not happy,<built-in method strip of str object at 0x0000...
2,I booked this hotel through Hotwire at the low...,not happy,<built-in method strip of str object at 0x0000...
3,Stayed here with husband and sons on the way t...,happy,<built-in method strip of str object at 0x0000...
4,My girlfriends and I stayed here to celebrate ...,not happy,<built-in method strip of str object at 0x0000...


In [50]:
df.describe()

,Description,Is_Response
count,38932,38932
unique,38932,2
top,Wow! I stay in a lot of hotels and this hotel ...,happy
freq,1,26521


In [59]:
data = train

In [54]:
data["Is_Response"] = data["Is_Response"].map({"happy" : "positive", "not happy" : "negative"})
data.sample(3)


,Description,Is_Response
24925,We stayed at the Andaz on -th Avenue for - nig...,positive
31836,"We booked our stay through Priceline. First, t...",negative
13684,We stayed at the Hyatt Capitol Hill from Frida...,positive


In [60]:
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()
twitter_handle = r'@[A-Za-z0-9_]+'                         # remove twitter handle (@username)
url_handle = r'http[^ ]+'                                  # remove website URLs that start with 'https?://'
combined_handle = r'|'.join((twitter_handle, url_handle))  # join
www_handle = r'www.[^ ]+'                                  # remove website URLs that start with 'www.'
punctuation_handle = r'\W+'

In [61]:
stop_words = set(nltk.corpus.stopwords.words('english'))

In [62]:
def process_text(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()

    try:
        text = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        text = souped

    cleaned_text = re.sub(punctuation_handle, " ",(re.sub(www_handle, '', re.sub(combined_handle, '', text)).lower()))
    cleaned_text = ' '.join([word for word in cleaned_text.split() if word not in stop_words])

    return (" ".join([word for word in tokenizer.tokenize(cleaned_text) if len(word) > 1])).strip()

In [65]:
cleaned_text = []

for text in data.Description:
    cleaned_text.append(process_text(text))

clean_text = pd.DataFrame({'clean_text' : cleaned_text})
data = pd.concat([data, clean_text], axis = 1)

data.sample(5)

,Description,Is_Response,clean_text
5660,Wish our flights could have been as relaxing a...,positive,wish flights could relaxing wonderful humphrey...
25643,Stayed at La Quinta Austin North with my famil...,negative,stayed la quinta austin north family yorkie ex...
2328,Room: Stained carpet from a previous ceiling l...,negative,room stained carpet previous ceiling leak unwa...
14977,We stayed at the comfort inn for these last --...,positive,stayed comfort inn last days chose read good r...
38305,"In late September-early October, we stayed - n...",positive,late september early october stayed nights har...


In [112]:
x = data['clean_text']
y = data['Is_Response']

In [121]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
x_train,x_test, y_train, y_test = train_test_split(x,y,test_size = 0.1, random_state = 47)

In [118]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


In [119]:
tvec = TfidfVectorizer()
clf2 = LogisticRegression()
rfc = RandomForestClassifier()

In [133]:
from sklearn.pipeline import Pipeline
model = Pipeline([('vectorizer',tvec),('classifier',clf2)])
model.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [134]:
pred = model.predict(x_test)
print(accuracy_score(y_test,pred))


0.8787878787878788


In [135]:
print(confusion_matrix(y_test,pred))

[[ 947  297]
 [ 175 2475]]


In [136]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

    negative       0.84      0.76      0.80      1244
    positive       0.89      0.93      0.91      2650

   micro avg       0.88      0.88      0.88      3894
   macro avg       0.87      0.85      0.86      3894
weighted avg       0.88      0.88      0.88      3894



In [137]:
model_rfc = Pipeline([('vectorizer',tvec),('classifier',rfc)])
model_rfc.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [138]:
pred = model.predict(x_test)
print(accuracy_score(y_test,pred))

0.8787878787878788


In [139]:
print(confusion_matrix(y_test,pred))

[[ 947  297]
 [ 175 2475]]


In [140]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

    negative       0.84      0.76      0.80      1244
    positive       0.89      0.93      0.91      2650

   micro avg       0.88      0.88      0.88      3894
   macro avg       0.87      0.85      0.86      3894
weighted avg       0.88      0.88      0.88      3894



In [143]:
data_test = test
data_test.head()

,User_ID,Description,Browser_Used,Device_Used
0,id80132,Looking for a motel in close proximity to TV t...,Firefox,Mobile
1,id80133,Walking distance to Madison Square Garden and ...,InternetExplorer,Desktop
2,id80134,Visited Seattle on business. Spent - nights in...,IE,Tablet
3,id80135,This hotel location is excellent and the rooms...,Edge,Mobile
4,id80136,This hotel is awesome I love the service Antho...,Mozilla,Mobile


In [145]:
data_test.drop(columns = ['User_ID', 'Browser_Used', 'Device_Used'],inplace = True)
data_test.head()

,Description
0,Looking for a motel in close proximity to TV t...
1,Walking distance to Madison Square Garden and ...
2,Visited Seattle on business. Spent - nights in...
3,This hotel location is excellent and the rooms...
4,This hotel is awesome I love the service Antho...


In [146]:
cleaned_text = []

for text in data_test.Description:
    cleaned_text.append(process_text(text))

clean_text = pd.DataFrame({'clean_text' : cleaned_text})
data_test = pd.concat([data_test, clean_text], axis = 1)

data_test.sample(5)

,Description,clean_text
7549,I know you had constuction going on the first ...,know constuction going first floor another flo...
5952,"Very nice hotel with perfect location, you can...",nice hotel perfect location cannot wrong booki...
4886,I have one word ... RUN! This property is old ...,one word run property old outdated room dark d...
26386,"Location is good, staff very nice, rooms small...",location good staff nice rooms small clean enj...
14173,I have been meaning to write this for some wee...,meaning write weeks per usual life got way sta...


In [148]:
data_test['result'] = model.predict(data_test["clean_text"])
data_test.head()

,Description,clean_text,result
0,Looking for a motel in close proximity to TV t...,looking motel close proximity tv taping dr phi...,negative
1,Walking distance to Madison Square Garden and ...,walking distance madison square garden penn su...,positive
2,Visited Seattle on business. Spent - nights in...,visited seattle business spent nights vintage ...,positive
3,This hotel location is excellent and the rooms...,hotel location excellent rooms clean suite one...,negative
4,This hotel is awesome I love the service Antho...,hotel awesome love service anthony really grea...,positive
